In [1]:
from bs4 import BeautifulSoup
import urllib2
import requests
import pandas as pd
import re
import time
import numpy as np
import json

In [6]:
all_cand = "http://myneta.info/ls2014/index.php?action=summary&subAction=candidates_analyzed&sort=candidate#summary"
winners_crim = "http://myneta.info/ls2014/index.php?action=summary&subAction=winner_crime&sort=candidate#summary"
source=requests.get(all_cand) # Modify here: winners works with smaller sample, all_cand with the whole set

In [7]:
tree= BeautifulSoup(source.text,"html.parser")

In [8]:
len(tree)

22

In [9]:
tree.title

<title>Total candidates analyzed by NEW:Loksabha 2014 Election</title>

In [10]:
all_links = tree.find_all('a')

In [11]:
table_pol = tree.findAll('table')[2]
rows = table_pol.findAll("tr")[2:]
print rows[10:13]

[<tr><td>11</td><td><a href="candidate.php?candidate_id=1105">A Prasad</a><b></b></td><td>KOLAR</td>
<td>IND</td><td align="center" style="">0</td><td>10th Pass</td>
<td align="right">Rs 8,40,000<br><span style="color:black;font-size:70%;text-decoration:none;white-space:nowrap"> ~ 8 Lacs+</span></br></td><td align="right">Rs 0<br><span style="color:black;font-size:70%;text-decoration:none;white-space:nowrap"> ~ </span> </br></td></tr>, <tr><td>12</td><td><a href="candidate.php?candidate_id=340">A Sampath</a><b></b></td><td>ATTINGAL</td>
<td>CPI(M)</td><td align="center" style=""><span style="color:red;font-weight:bold;font-size:150%">1</span></td><td>Doctorate</td>
<td align="right">Rs 1,77,91,149<br><span style="color:black;font-size:70%;text-decoration:none;white-space:nowrap"> ~ 1 Crore+</span></br></td><td align="right">Rs 14,21,392<br><span style="color:black;font-size:70%;text-decoration:none;white-space:nowrap"> ~ 14 Lacs+</span> </br></td></tr>, <tr><td>13</td><td><a href="cand

In [12]:
# We build one dictionary per candidate #
# We give each candidates to idenfiers: its order in the list and its url address

def id2(r):
    url_string = str(r.find("a").get("href"))
    id2 = int(re.search(r'\d+', url_string).group())
    return id2

def c_link(r):
    return r.find("a").get("href")
def name(r):
    return r.find("a").get_text()
def cols(r):
    return r.findAll("td")
def assets(r):
    col = cols(r)
    ass1 = col[6].get_text().split("~")[0].encode('ascii', 'ignore').replace("Rs","").replace(",","")
    if ass1 == "Nil":
        ass2 = 0
    else:
        ass2=int(ass1)
    return ass2

def liab(r):
    col = cols(r)
    liab1 = col[7].get_text().split("~")[0].encode('ascii', 'ignore').replace("Rs","").replace(",","")
    if liab1 == "Nil":
        liab2 = 0
    else:
        liab2 = int(liab1)
    return liab2

info_candidate = lambda r: [int(cols(r)[0].get_text()),id2(r), r.find("a").get("href"),r.find("a").get_text(),
                            cols(r)[2].get_text(),cols(r)[3].get_text(),cols(r)[5].get_text(),
                            int(cols(r)[4].get_text()),assets(r), liab(r)]

title = ['id','id2','url','name','district','party','education','nr_crime','assets','liabilities']
dict_candidates = [dict(zip(title,info_candidate(r))) for r in rows]

In [13]:
len(dict_candidates)
dict_candidates[0:5]

[{'assets': 154566136,
  'district': u'NAWADA',
  'education': u'Post Graduate',
  'id': 1,
  'id2': 148,
  'liabilities': 2604969,
  'name': u'Kaushal Yadav',
  'nr_crime': 8,
  'party': u'JD(U)',
  'url': u'candidate.php?candidate_id=148'},
 {'assets': 2191523,
  'district': u'AZAMGARH',
  'education': u'Others',
  'id': 2,
  'id2': 9496,
  'liabilities': 0,
  'name': u'M. Aamir Rashadi',
  'nr_crime': 1,
  'party': u'Rashtriya Ulama Council',
  'url': u'candidate.php?candidate_id=9496'},
 {'assets': 306023,
  'district': u'MAHARAJGANJ',
  'education': u'Graduate Professional',
  'id': 3,
  'id2': 9706,
  'liabilities': 0,
  'name': u'Rakesh Kumar Giri',
  'nr_crime': 0,
  'party': u'IND',
  'url': u'candidate.php?candidate_id=9706'},
 {'assets': 3630000,
  'district': u'CHENNAI SOUTH',
  'education': u'8th Pass',
  'id': 4,
  'id2': 6912,
  'liabilities': 850000,
  'name': u'(Kuppal)G.Devadoss',
  'nr_crime': 0,
  'party': u'IND',
  'url': u'candidate.php?candidate_id=6912'},
 {'ass

In [14]:
# Now we create a really big dictionary which stores url and page for each candidate
# Work in progress...
# First transform the thing into a dataframe

df_pol = pd.DataFrame(dict_candidates)
order_cols = ['id2','name','district','party','education','assets','liabilities','nr_crime','url']
df_pol = df_pol[order_cols].sort(['assets'],ascending=0)

df_small=df_pol[["id2","url"]][0:3] # I use df_small to test the code before running it on 8400 candidates!
df_small
df_pol.head()

#Note already that the same guy can be candidate at 2 districts #
# Might be a problem

,id2,name,district,party,education,assets,liabilities,nr_crime,url
4522,611,Nandan Nilekani,BANGALORE SOUTH,INC,Graduate Professional,77102957219,0,0,candidate.php?candidate_id=611
6782,9403,Shamali Das,KOLKATA DAKSHIN,IND,8th Pass,20000060000,0,0,candidate.php?candidate_id=9403
6781,9378,Shamali Das,JADAVPUR,IND,8th Pass,20000060000,0,0,candidate.php?candidate_id=9378
433,2677,Anil Kumar Sharma,JAHANABAD,JD(U),Doctorate,8498803720,1095393904,1,candidate.php?candidate_id=2677
2846,8183,Jayadev Galla,GUNTUR,TDP,Graduate,6830581361,202334283,0,candidate.php?candidate_id=8183


In [51]:
urlcache={}

In [52]:
def get_page(url):
    # Check if URL has already been visited.
    if (url not in urlcache) or (urlcache[url]==1) or (urlcache[url]==2):
        time.sleep(1)
        steps = len(urlcache)
        if 100*int(steps/100)==steps:
            print steps # This counter tells us how many links were downloaded at every 100 mark
        # try/except blocks are used whenever the code could generate an exception (e.g. division by zero).
        # In this case we don't know if the page really exists, or even if it does, if we'll be able to reach it.
        try:
            r = requests.get("http://myneta.info/ls2014/%s" % url)

            if r.status_code == 200:
                urlcache[url] = r.text
            else:
                urlcache[url] = 1
        except:
            urlcache[url] = 2
    return urlcache[url]

In [57]:
df_pol["url"].apply(get_page) # This is a very long call (~4.5 hours on full dataset)
                              # I am saving it in order to run it only once

NameError: global name 'urlcache' is not defined

In [54]:
print np.sum([(urlcache[k]==1) or (urlcache[k]==2) for k in urlcache])# no one or 0's
print len(df_pol.url.unique())==len(urlcache)#we got all of the urls

7
True


In [55]:
with open("tempdata/polinfo.json","w") as fd:
    json.dump(urlcache, fd)
del urlcache

In [56]:
with open("tempdata/polinfo.json") as json_file:
    pol_pages = json.load(json_file) # Next iterations will start from here!

In [58]:
# Now each candidate has an url - so later we  will parse each of their pages and scrape them#
#But first let's look how the page looks like for a few individual candidates
pages_list = [pol_pages[x] for x in df_pol['url'][0:10]] # List of 10 that we can start looking at

i = 5
url_candidate = pol_pages.keys()[i]
full_url = "http://myneta.info/ls2014/" + url_candidate
page_candidate = pol_pages.values()[i]
c_soup = BeautifulSoup(page_candidate,"html.parser")
print full_url

http://myneta.info/ls2014/candidate.php?candidate_id=4794


In [59]:
#### Personal info #########

personal = c_soup.findAll(attrs={"class": "grid_3 alpha"})[0]
full_name = personal.find("h2").get_text()
district1 = personal.find("h5").get_text()
grid2 = personal.findAll(attrs={"class":"grid_2 alpha"})
party_full = grid2[0].get_text().split(":")[1]
age = int(grid2[2].get_text().split(":")[1])
address = grid2[3].get_text().split(":")[1].split("\n")[1] # Careful this one changes
self_profession = personal.find("p").get_text().split('\n')[0].split(":")[1]
spouse_profession = personal.find("p").get_text().split('\n')[1].split(":")[1]
print full_name, "\n works as a:", self_profession, ". \n Profession of spouse:", spouse_profession, ". \n Address: ",
print address, ", in district: ", district1, "for party:", party_full, "The dude is", age, "years old"

########## Assets  & Criminality ###########
# Ok let's get serious and find his assets and criminal activity
all_tables = c_soup.findAll("table")
print "\n \n" , len(all_tables)
table_inc = all_tables[0]
table_crim = all_tables[1]
table_assets1 = all_tables[2]
table_assets2 = all_tables[3]
table_liab = all_tables[4]
print table_inc
# That does not really work: some have a table for criminal activities, others not
# So we need more work to know exactly how to have the right table correspond to the right thing


PARAS DADA SAKLECHA  
 works as a:  . 
 Profession of spouse: Interest &  Job work . 
 Address:              H. NO. 19/1 Shanti nagar Shatlam  , in district:  
        MANDSOUR  (MADHYA PRADESH)
         for party: AAP
  The dude is 59 years old

 
9
<table cellpadding="0" cellspacing="0" id="table3" width="80%">
<tr style="background-color:#336633;color:white;" valign="top"><th colspan="3">Other Elections</th></tr>
<tr style="background-color:#336633;color:white;" valign="top"><th>Declaration in</th><th>Declared Assets</th><th>Declared Cases</th></tr>
<tr><td style="background-color:#F3F3F3;"><b>Madhya Pradesh 2013</b></td> <td style="background-color:#F3F3F3;"><b>Rs46,24,553</b><span style="color:black;text-decoration:none;white-space:nowrap"><br>~46 Lacs+</br></span></td><td style="background-color:#F3F3F3;">0</td>
</tr><tr><td style="background-color:#F3F3F3;"><b>Madhya Pradesh 2008</b></td> <td style="background-color:#F3F3F3;"><b>Rs9,45,000</b><span style="color:black;text-decora